# A quantum solution to the Discrete Logarithm Problem

*Qiskit Summer Jam, North Carolina, 2020*

*Matthew Gregoire, Richard Howe, Guan-Wen Chou, Anton Nikulsin, and Jessica Horn*

## The Discrete Logarithm Problem

In general terms, let $G$ be a group, let $a$ be an element of $G$ of order $r$, and let $b$ be some power of $a$, so that $b = a^m$, for $0 \leq m < r$. The discrete logarithm problem is as follows: given $a$ and $b$, find the integer $m$. While $b$ may be easy to calculate given $a$ and $m$, there is no known efficient classical algorithm for finding $m$ given $a$ and $b$. This fact is the basis of many modern cryptosystems. In this project, we present a novel method for solving the discrete logarithm problem.

## Oracle-based Algorithm and the Half-Bit

The *half-bit* of $b$, denoted $HB_a(b)$, is defined as follows.

$$ HB_a(b) = \begin{cases}
0 & 0 \leq m < r/2 \\
1 & r/2 \leq m < r
\end{cases}
$$

Thus the half-bit of $b$ has a natural interpretation as the most significant bit of the binary expansion of $m$. In his 1988 thesis, Burton S. Kaliski Jr. proves that, given an oracle that is able to correctly predict the half-bit with probability at least $1/2 + \epsilon$, the discrete logarithm problem can be solved in polynomial time, by making several oracle queries on values closely related to $b$. Kaliski also presents the algorithm to solve this problem.

In 2017, Kaliski published a paper describing a quantum circuit to efficiently implement this oracle, which he calls a quantum "magic box." While Shor's algorithm depends on solving the entire discrete logarithm problem using one quantum circuit, implementing this oracle would only require the quantum processor to calculate one bit of the discrete logarithm at a time, potentially saving valuable computational resources.

## Our Goal

In this project, our first and foremost goal was to implement the quantum "magic box" in Qiskit. We also 